In [1]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [3]:
loader = CSVLoader(
  file_path = "sample_data/울산광역시_동구_작은도서관_대출현황관리_20250306.csv",
  encoding = "cp949"
)

In [4]:
documents = loader.load()
print(f"Loaded {len(documents)} documents from CSV.")

Loaded 17983 documents from CSV.


In [5]:
print("첫 번째 문서 내용 일부:")
print(documents[0].page_content[:200] + "...")

첫 번째 문서 내용 일부:
대출일: 2025-02-08 15:38:00
반납일: 2025-02-20 09:35:00
분류기호: 813.7
등록번호: HA0000028556
서명: 소년이 온다 : 한강 장편소설
작가: 한강 지음
출판사: 창비
연체일수: 0
작업장: KLAS
작성일자: 2025-02-08
기준일자: 2025-03-06...


In [6]:
text_splitter = CharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=100
)

docs = text_splitter.split_documents(documents)

print(f"Split into {len(docs)} text chunks")

Split into 17983 text chunks


In [7]:
print(docs[0].page_content[:200] + "...")

대출일: 2025-02-08 15:38:00
반납일: 2025-02-20 09:35:00
분류기호: 813.7
등록번호: HA0000028556
서명: 소년이 온다 : 한강 장편소설
작가: 한강 지음
출판사: 창비
연체일수: 0
작업장: KLAS
작성일자: 2025-02-08
기준일자: 2025-03-06...


In [ ]:
embeddings = OpenAIEmbeddings()

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_90083/2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [9]:
vector_store = FAISS.from_documents(docs, embeddings)

In [10]:
print(f"Total vectors: {vector_store.index.ntotal}")

Total vectors: 17983


In [11]:
query = "(흔한남매) 수수께끼 어드벤처. 5 도서의 작가는 누구야?"

In [12]:
results = vector_store.similarity_search(query, k=3)

In [13]:
if len(results) == 0:
  print("관련 데이터를 찾지 못했습니다.")
else:
  print(f"Found {len(results)} relevant pieces of data.")

  for i, res in enumerate(results, start=1):
    snippet = res.page_content[:100].replace("\n", " ")
    print(f"{i}. {snippet}...")

Found 3 relevant pieces of data.
1. 대출일: 2025-02-28 11:22:00 반납일:  분류기호: 31 등록번호: DN0000029651 서명: (흔한남매) 수수께끼 어드벤처. 5 작가: 흔한남매 원작 ; 노지영...
2. 대출일: 2025-02-26 17:23:00 반납일:  분류기호: 388.6 등록번호: AM0000023448 서명: (흔한남매) 수수께끼 어드벤처. 5 작가: 흔한남매 원작 ; ...
3. 대출일: 2025-02-18 15:24:00 반납일:  분류기호: 31 등록번호: DN0000026908 서명: (흔한남매) 수수께끼 어드벤처. 4 작가: 흔한남매 원작 ; 노지영...


In [14]:
llm = OpenAI(
  temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=vector_store.as_retriever()
)

answer = qa_chain.run(query)

print("Answer: ", answer)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_90083/805822090.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(
/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_90083/805822090.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Answer:   흔한남매 원작 ; 노지영 글 ; 도니패밀리 그림


In [15]:
memory = ConversationBufferMemory(
  memory_key='chat_history',
  return_messages=True
)

conv_chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  retriever=vector_store.as_retriever(),
  memory=memory
)

result1 = conv_chain.invoke({"question": "(흔한남매) 수수께끼 어드벤처. 5 도서의 작가는 누구야?"})
print("Answer 1: ", result1['answer'])

result2 = conv_chain.invoke({"question": "출판사는 어디야?"})
print("Answer 2: ", result2['answer'])

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_90083/2919174396.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Answer 1:   흔한남매 원작 ; 노지영 글 ; 도니패밀리 그림
Answer 2:   미래엔아이세움
